**대단한 점수와 실력이 아니지만**   
**`단일 모델`로 `정규화`, `로그화` 없이 유의미한 결과를 내본 저의 경험을 공유합니다.**
* 특성은 date와 zipcode 만 건드렸습니다.
* PCA로 위도경도 묶어서 2개, 전체데이터 묶어서 2개, 총4개 추가했습니다.
* 스케일링, 노멀라이징 모두 **뺐**습니다. (평가결과가 훨씬 더 높았습니다)
* XGB 단일 모델 사용했습니다.

# 1. 진행방법
1. 최초 xgb 모델 기준으로 데이터 특징이 변경되거나 추가변수(파생변수) 나올 때마다 평가 후 적용했습니다.
2. 괜찮으면 적용시키고 안 좋으면 윗단 아랫단 작업순서를 바꿔가면서 평가했습니다.
3. 어느 정도 점수가 올라왔을 때 모델 파라미터를 GridSearch와 수작업(?)으로 찾아다녔습니다.
4. 모델은 XGB, LGBM, GBR 세 가지를 비교 분석했습니다.
    * XGBoost
    * LGBM
    * GradientBoostingRegression

----
# 2. 결론정리
## 1. 특성
### Date
* 특성은 `object` 타입이라 바꿔줘야 했던 `date`를 `년도`와 `달`별로 쪼개서 추가했습니다.
* 평가결과 `118848` 까지 왔습니다.

### 지역정보 zipcode
* 상식적으로 집값에 핵심은 지역이란 생각으로 접근했습니다.
* `zipcode` 별로 묶어서 카테고리 0 ~ 10까지 특성을 추가했었는데 [비슷하고 또 더 깔끔하게 정리하신 허진명님 커널 참고](https://www.kaggle.com/tmheo74/geo-data-eda-and-feature-engineering#PCA-Transformation---Lat,-Long)했습니다.
* 평가결과 `116464` 까지 왔습니다.

## 2. 변형
당연하듯 진행해왔던 통계적 분석과정들에 대해 평가점수가 좋지 않아 과감히 뺐습니다.

### Scaler
* 특이하게도 스케일링 하니까 점수가 나빴습니다.
* `MinMaxScaler`, `MaxAbsScaler`, `StandardScaler`, `RobustScaler` 모두 비교분석했습니다.
* `MaxAbsScaler` 가 점수가 미비하게 좋았지만 나중에 후반으로 가면서 비슷해져 모두 제외했습니다.
### log
* 분포가 정규분포그래프로 멋지게 그려지지 않고 쏠려있으면 대부분 log를 씌워서 그래프를 이쁘게 만드는데
* 평가결과가 훨씬 좋지 못했습니다.
* 수치형 특성들에 대해서 또 크게 쏠려있는 특성들에 대해서 이리저리 시도했지만 마찬가지로 별로였습니다.

두 과정 모두 순서도 바꿔보고 이리저리 해봤지만
* 평가결과 `180000` 까지 나빠져 과감히 제외했습니다.

## 3. PCA
### 지역정보 lat, long
* 위에서와 마찬가지로 집값의 핵심은 지역이란 생각에 PCA로 클러스터링 접근했습니다. 
* 더 깔끔하게 정리하신 [허진명님 커널 참고하여 추가했습니다.](https://www.kaggle.com/tmheo74/geo-data-eda-and-feature-engineering#PCA-Transformation---Lat,-Long)
* 평가결과 2000 정도 좋아졌습니다.

### 전항목 PCA 
* 전 특성을 PCA 클러스터링을 통해 차원축소를 했습니다.
* 2차원부터 18차원까지 비교했는데 2차원부터는 상승폭이 크지 않아 2차원으로 축소해서 추가했습니다.
* 평가결과 2000 정도 좋아졌습니다.

## 4. 모델
### 속성
음.. 이걸 어떻게 설명을..... 대부분 이거 찾는데 컴퓨터 자원을 많이 쏟은거 같습니다. 그리드서치로 너무 오래걸렸고 그래서 수작업 한땀한땀 찾으러 다니기도 했고.. 그냥 좋다는거 남들 다 하는거, 그리고 상식적으로 좋아질거 같은거 다 해봤습니다.
* 당연히 높으면 좋을꺼라 생각했던 속성에 대해서 다시금 생각하게 됐습니다. 

```python
num_rounds = 20000
ealry_stopping = 500
verbose_eval = 300

xgb_params_add1 ={
    'seed': RANDOM_SEED,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.4,
    'silent': True,
    'n_estimators':5000}
```


### 모델
* 평가결과 `XGB` >> `LGBM` > `GBR`
* Average CV-Score
  * 111400.48654607737 - `XGB`
  * 115003.96471292987 - `LGBM`
  * 116762.99081471628 - `GBR`
* XGB가 월등히 높진 않았지만 그래도 역시나 높았습니다.


## 5. seed
* 개인적으로 다른 곳에서 억지로 끌어올리던 점수를 `seed` 하나바꿈으로 결과가 확확 바뀌었습니다.
* 점수 4000, 5000 이 오르락 내리락..
* 그래서 seed 값도 평가점수 높은 걸 찾기위해 수작업했습니다. ㅎㅎ
* 다시 말씀드리지만 저는 seed로 높아졌을뿐, 그저 제 리더보드 순위는 우연이었을뿐..
* 정말 최고사랑 631.... 제 커널, 제 순위의 핵심.. 비기.. 
```python
RANDOM_SEED = 631
np.random.seed(RANDOM_SEED)
```

----
# 3. 과정

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
!ls ../input

sample_submission.csv  test.csv  train.csv


불러오기

In [3]:
train = pd.read_csv('../input/train.csv', index_col=0)
test = pd.read_csv('../input/test.csv', index_col=0)

특성 type 확인해서 정리할거 확인

In [4]:
for i in test.columns:
    tyty = test[i].dtype    
    if not tyty == 'int64' and not tyty == 'float64':
        print(f'feature: {i}, type: {str(tyty)}')

feature: date, type: object


date 특성 년과 달로 쪼개주고 int 타입으로 바꿔주고 삭제

In [5]:
def clean_data(dataset):
    dataset['data_y'] = ''
    dataset['data_m'] = ''
    dataset['data_y'] = dataset['date'].apply(lambda x : str(x[:4])).astype(int)
    dataset['data_m'] = dataset['date'].apply(lambda x : str(x[4:6])).astype(int)
    dataset.drop('date', axis=1, inplace=True)
    return dataset

zipcode 추가

In [6]:
def geogege(data):
    data['zipcode'] = data['zipcode'].astype(str)  
    data['zipcode-3'] = data['zipcode'].apply(lambda x : str(x[2:3])).astype(int)
    data['zipcode-4'] = data['zipcode'].apply(lambda x : str(x[3:4])).astype(int)
    data['zipcode-5'] = data['zipcode'].apply(lambda x : str(x[4:5])).astype(int)
    data['zipcode-34'] = data['zipcode'].apply(lambda x : str(x[2:4])).astype(int)
    data['zipcode-45'] = data['zipcode'].apply(lambda x : str(x[3:5])).astype(int)
    data['zipcode-35'] = data['zipcode'].apply(lambda x : str(x[2:5])).astype(int)
    data.drop('zipcode', axis=1, inplace=True)
    return data

PCA 특성2개 추가

In [7]:
def ppcca1(trainset, testset):
    pca2 = PCA(n_components=2)
    coord = trainset[['lat','long']]
    coord_test = testset[['lat','long']]
    
    principalComponents_updated = pca2.fit_transform(coord)
    trainset['coord_pca1']= ''
    trainset['coord_pca2']= ''
    trainset['coord_pca1']= principalComponents_updated[:, 0]
    trainset['coord_pca2']= principalComponents_updated[:, 1]

    principalComponents_updated_test = pca2.transform(coord_test)
    testset['coord_pca1']= ''
    testset['coord_pca2']= ''
    testset['coord_pca1']= principalComponents_updated_test[:, 0]
    testset['coord_pca2']= principalComponents_updated_test[:, 1]
    return trainset, testset

PCA 모든 데이터에 대해서. 

In [8]:
def ppcca2(trainset, testset):
    pca1 = PCA(n_components=2)

    principalComponents_updated = pca1.fit_transform(trainset)
    trainset['pca1']= ''
    trainset['pca2']= ''
    trainset['pca1']= principalComponents_updated[:, 0]
    trainset['pca2']= principalComponents_updated[:, 1]

    principalComponents_updated_test = pca1.transform(testset)
    testset['pca1']= ''
    testset['pca2']= ''
    testset['pca1']= principalComponents_updated_test[:, 0]
    testset['pca2']= principalComponents_updated_test[:, 1]
    return trainset, testset

사용한 모델 파라메터는

In [9]:
RANDOM_SEED = 631
np.random.seed(RANDOM_SEED)

num_rounds = 20000
ealry_stopping = 500
verbose_eval = 300

xgb_params_add1 ={
    'seed': RANDOM_SEED,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.4,
    'silent': True,
    'n_estimators':5000,
}

이렇게 했더니 결과가 20위권으로...

전체적으로 그래프랑 도표, 그리고 파라메터 찾아가는 과정은 차차 업뎃 하겠습니다. 너무 이것저것 건드리고 널려놓은게 많아서 정리가 필요합니다.

여기까지 봐주셔서 감사합니다.
다시 말씀드리지만 저의 비법은 `RANDOM_SEED = 631` 입니다 ^^ 